In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.metrics import mean_squared_error

data = pd.read_csv('earthquake_data.csv')



In [2]:
data.head()

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
0,"M 7.0 - 18 km SW of Malango, Solomon Islands",7.0,22-11-2022 02:03,8,7,green,1,768,us,117,0.509,17.0,mww,14.000,-9.7963,159.596,"Malango, Solomon Islands",Oceania,Solomon Islands
1,"M 6.9 - 204 km SW of Bengkulu, Indonesia",6.9,18-11-2022 13:37,4,4,green,0,735,us,99,2.229,34.0,mww,25.000,-4.9559,100.738,"Bengkulu, Indonesia",NaN,NaN
2,M 7.0 -,7.0,12-11-2022 07:09,3,3,green,1,755,us,147,3.125,18.0,mww,579.000,-20.0508,-178.346,NaN,Oceania,Fiji
3,"M 7.3 - 205 km ESE of Neiafu, Tonga",7.3,11-11-2022 10:48,5,5,green,1,833,us,149,1.865,21.0,mww,37.000,-19.2918,-172.129,"Neiafu, Tonga",NaN,NaN
4,M 6.6 -,6.6,09-11-2022 10:14,0,2,green,1,670,us,131,4.998,27.0,mww,624.464,-25.5948,178.278,NaN,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      782 non-null    object 
 1   magnitude  782 non-null    float64
 2   date_time  782 non-null    object 
 3   cdi        782 non-null    int64  
 4   mmi        782 non-null    int64  
 5   alert      415 non-null    object 
 6   tsunami    782 non-null    int64  
 7   sig        782 non-null    int64  
 8   net        782 non-null    object 
 9   nst        782 non-null    int64  
 10  dmin       782 non-null    float64
 11  gap        782 non-null    float64
 12  magType    782 non-null    object 
 13  depth      782 non-null    float64
 14  latitude   782 non-null    float64
 15  longitude  782 non-null    float64
 16  location   777 non-null    object 
 17  continent  206 non-null    object 
 18  country    484 non-null    object 
dtypes: float64(6), int64(5), object(8)
memory usage: 1

In [4]:
data=data.dropna()

In [5]:

X = data.drop(['magnitude'], axis=1)  
y = data['magnitude']  



In [6]:
X = X.drop('date_time', axis=1)
X=X.drop('location',axis=1)
X=X.drop('continent',axis=1)
X=X.drop('country',axis=1)
X=X.drop('title',axis=1)


In [7]:
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns


In [8]:
print(categorical_cols)

Index(['alert', 'net', 'magType'], dtype='object')


In [9]:
X.columns

Index(['cdi', 'mmi', 'alert', 'tsunami', 'sig', 'net', 'nst', 'dmin', 'gap',
       'magType', 'depth', 'latitude', 'longitude'],
      dtype='object')

In [ ]:
import pickle
label_encoder = LabelEncoder()

categorical_cols = ['alert', 'net']

for col in categorical_cols:
    label_encoder = LabelEncoder()
    X[col] = label_encoder.fit_transform(X[col])

    with open(f'label_encoder_{col}.pkl', 'wb') as le_file:
        pickle.dump(label_encoder, le_file)

print("Encoded DataFrame:")
print(X.head())


In [11]:
print(X)

     cdi  mmi  alert  tsunami   sig  net  nst    dmin    gap magType    depth  \
0      8    7      0        1   768    3  117  0.5090   17.0     mww   14.000   
8      8    7      3        1  1179    3  175  2.1370   92.0     mww   20.000   
9      9    8      3        1  1799    3  271  1.1530   69.0     mww   26.943   
10     9    9      3        1   887    3  215  0.4010   34.0     mww   10.000   
11     7    7      0        1   756    3  178  0.4300   54.0     mww   10.000   
..   ...  ...    ...      ...   ...  ...  ...     ...    ...     ...      ...   
406    0    7      0        1   670    3  697  0.0000   15.3     mww   11.000   
408    5    5      0        1   904    3  562  0.0000   23.7     mww  145.000   
415    6    7      3        0   814    3  686  0.0000   28.0     mww  107.000   
416    6    7      0        0   771    3  596  0.0000   19.3     mww   45.000   
507    9    9      2        0  2910    1   10  0.5137  239.0      mw    9.987   

     latitude  longitude  


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(f_regression, k=10)
X_new = selector.fit_transform(X, y)

selected_features_mask = selector.get_support()

selected_feature_names = X.columns[selected_features_mask]

print("Selected features:", selected_feature_names.tolist())
print("Selected features shape:", X_new.shape)


Selected features: ['cdi', 'mmi', 'alert', 'tsunami', 'sig', 'net', 'dmin', 'gap', 'depth', 'longitude']
Selected features shape: (96, 10)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

dt_model = DecisionTreeRegressor(random_state=42)
rf_model = RandomForestRegressor(random_state=42)
lr_model = LinearRegression()

dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)
dt_mse = mean_squared_error(y_test, dt_predictions)

rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_predictions)

lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)
lr_mse = mean_squared_error(y_test, lr_predictions)

print("Mean Squared Error for Decision Tree Regressor:", dt_mse)
print("Mean Squared Error for Random Forest Regressor:", rf_mse)
print("Mean Squared Error for Linear Regression:", lr_mse)

Mean Squared Error for Decision Tree Regressor: 0.14550000000000002
Mean Squared Error for Random Forest Regressor: 0.07419844999999994
Mean Squared Error for Linear Regression: 0.11114795925097717


In [ ]:

results = pd.DataFrame(X_test, columns=selected_feature_names)
results['magnitude'] = y_test.reset_index(drop=True)

results.to_csv('X_test.csv', index=False)

print("X_test with target label 'magnitude' saved to 'X_test_with_magnitude.csv'.")


X_test with target label 'magnitude' saved to 'X_test_with_magnitude.csv'.


In [98]:
def categorize_magnitude(magnitude):
    if magnitude < 4.0:
        return 'low'
    elif 4.0 <= magnitude < 6.0:
        return 'medium'
    else:
        return 'high'


In [ ]:
processed_input = [[6.0,7.0,0.0,1.0,723.0,3.0,0.636,31.0,27.0,-71.5215]]
dt_prediction = dt_model.predict(processed_input)
rf_prediction = rf_model.predict(processed_input)
lr_prediction = lr_model.predict(processed_input)

print("Predicted Magnitude (Decision Tree):", dt_prediction[0])
print("Predicted Magnitude (Random Forest):", rf_prediction[0])
print("Predicted Magnitude (Linear Regression):", lr_prediction[0])

dt_category = categorize_magnitude(dt_prediction[0])
rf_category = categorize_magnitude(rf_prediction[0])
lr_category = categorize_magnitude(lr_prediction[0])

print("Magnitude Category (Decision Tree):", dt_category)
print("Magnitude Category (Random Forest):", rf_category)
print("Magnitude Category (Linear Regression):", lr_category)

Predicted Magnitude (Decision Tree): 6.6
Predicted Magnitude (Random Forest): 6.714000000000003
Predicted Magnitude (Linear Regression): 6.846835058102771
Magnitude Category (Decision Tree): high
Magnitude Category (Random Forest): high
Magnitude Category (Linear Regression): high


In [ ]:
with open('rfmodelE.pkl', 'wb') as rf_file:
    pickle.dump(rf_model, rf_file)

with open('label_encoderE.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)

print("Models and Label Encoder have been saved successfully.")


Models and Label Encoder have been saved successfully.
